### Import packages

In [1]:
library(GWmodel)      ### GW models
library(dplyr)
library(sp)           ## Data management
library(car)          ## vif
library(spdep)        ## Spatial autocorrelation
library(RColorBrewer) ## Visualization
library(classInt)     ## Class intervals
library(raster)       ## spatial data
library(grid)         # plot
library(gridExtra)    # Multiple plot
library(ggplot2)      # Multiple plot
library(gtable)
library(GGally)       # 相關係數圖矩陣（scatter plot matrix）
library(maptools)
library(MASS)
library(tmap)


載入需要的套件：maptools

載入需要的套件：sp

Checking rgeos availability: TRUE

載入需要的套件：robustbase

載入需要的套件：Rcpp

載入需要的套件：spatialreg

載入需要的套件：spData

載入需要的套件：Matrix

載入需要的套件：sf

Linking to GEOS 3.9.0, GDAL 3.2.1, PROJ 7.2.1

Welcome to GWmodel version 2.2-8.


載入套件：'dplyr'


下列物件被遮斷自 'package:stats':

    filter, lag


下列物件被遮斷自 'package:base':

    intersect, setdiff, setequal, union


載入需要的套件：carData


載入套件：'car'


下列物件被遮斷自 'package:dplyr':

    recode



載入套件：'spdep'


下列物件被遮斷自 'package:spatialreg':

    aple, aple.mc, aple.plot, as.spam.listw, as_dgRMatrix_listw,
    as_dsCMatrix_I, as_dsCMatrix_IrW, as_dsTMatrix_listw,
    can.be.simmed, cheb_setup, create_WX, do_ldet, eigen_pre_setup,
    eigen_setup, eigenw, errorsarlm, get.ClusterOption,
    get.coresOption, get.mcOption, get.VerboseOption,
    get.ZeroPolicyOption, GMargminImage, GMerrorsar, griffith_sone,
    gstsls, Hausman.test, impacts, intImpacts, invIrM, invIrW,
    Jacobian_W, jacobianSetup, l_max, lagmess, lagsarlm, lextrB,
    lextrS

### Function

#### pdf_plot

In [2]:

pdf_plot <- function(x){
  g = ggplot()+
    geom_histogram(aes(x = x, y = ..density..), 
                   fill = '#557C55', alpha = 0.8)+
    geom_density(aes(x = x, y = ..density..), 
                 color = '#062C30', size = 1)+
    theme_bw()
  
  return(g)
}

### Load datas

In [27]:
getwd()
path = '..\\..\\Roaming-Dogs-Data\\'
Variable_KS_df <- read.csv(paste0(path, "@Test_KS\\Variable.csv" ), fileEncoding = 'utf-8')
Variable_KS_df["Clinic"][is.na(Variable_KS_df["Clinic"])] = 0
Variable_KS_shp<-shapefile(paste0(path, "@Test_KS\\Variable.shp" ),encoding = 'big5')
Variable_KS_shp@data[is.na(Variable_KS_shp@data)] <- 0
Variable_KS_df[is.na(Variable_KS_df)] <- 0
crs(Variable_KS_shp) <- CRS('+init=EPSG:3826') 
# Variable_KS_shp_02 = st_read(paste0(path, "@Test_KS\\Variable.shp"))
# Variable_KS_centroid <-  st_centroid(Variable_KS_shp)
# colnames(Variable_KS_centroid)

[1] "c:/@@Roaming_Dogs_Data_Analysis/Roaming-Dogs-Code/Model-fitting"

Warning message in OGRSpatialRef(dsn, layer, morphFromESRI = morphFromESRI, dumpSRS = dumpSRS, :
"Discarded datum Taiwan_Datum_1997 in Proj4 definition: +proj=tmerc +lat_0=0 +lon_0=121 +k=0.9999 +x_0=250000 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs"
Warning message in showSRID(uprojargs, format = "PROJ", multiline = "NO", prefer_proj = prefer_proj):
"Discarded datum Taiwan_Datum_1997 in Proj4 definition"


### Create a residual dataframe

### select Xy

In [28]:
col_X = 
c( "Cluster", # 分群
"Market","Hospital" ,  "Temple",  "Ele" ,"Junior" ,"Senior", "Train.stat",  "Clinic", # 公共建設
"high_rat",   "mid_rat","low_rat", "M_F_RAT" , "P_DEN", "YOUN_DEP","OLD_DEP","AGING_IDX", # 人口統計(教育程度、人口密度...)
"Income_mea", "Income_med" ,"Income_Q1",  "Income_Q3",  "Income_sta") # 村里收入
col_y = c('Nt')

In [29]:
for(i in c(col_X, col_y)){

    Variable_KS_shp@data[i] = Variable_KS_df[i] = sapply(Variable_KS_df[i], function(x) as.numeric(x))
}

In [30]:
for(i in c('Market','Hospital',  "Temple",  "Ele" ,"Junior" ,"Senior", "Train.stat",  "Clinic")){
        
    i_new = paste0(i, "_den")
    print(i_new)
    
    Variable_KS_shp@data[i_new] = Variable_KS_shp@data[i]/Variable_KS_shp@data$Area_sqkm
    Variable_KS_df[i_new] = Variable_KS_df[i]/Variable_KS_df$Area_sqkm
}

[1] "Market_den"
[1] "Hospital_den"
[1] "Temple_den"
[1] "Ele_den"
[1] "Junior_den"
[1] "Senior_den"
[1] "Train.stat_den"
[1] "Clinic_den"


In [31]:
col_X = 
  c( "Cluster", # 分群
     "Market_den","Hospital_den" ,  "Temple_den",  "Ele_den" ,"Junior_den" ,"Senior_den", "Train.stat_den",  "Clinic_den", # 公共建設
     "high_rat",   "mid_rat","low_rat", "M_F_RAT" , "P_DEN", "YOUN_DEP","OLD_DEP","AGING_IDX", # 人口統計(教育程度、人口密度...)
     "Income_mea","Income_sta")

#### Normalize the data

In [32]:
col_X_scale = 
c( 
"Market_den","Hospital_den" ,  "Temple_den",  "Ele_den" ,"Junior_den" ,"Senior_den", "Train.stat_den",  "Clinic_den", # 公共建設
"high_rat",   "mid_rat","low_rat", "M_F_RAT" , "P_DEN", "YOUN_DEP","OLD_DEP","AGING_IDX", # 人口統計(教育程度、人口密度...)
"Income_mea", "Income_sta")

In [33]:
for (i in col_X_scale){
    print(i)

    Variable_KS_df[i] <- scale(Variable_KS_df[i])
    Variable_KS_shp@data[i] <-scale(Variable_KS_shp@data[i])

}


[1] "Market_den"
[1] "Hospital_den"
[1] "Temple_den"
[1] "Ele_den"
[1] "Junior_den"
[1] "Senior_den"
[1] "Train.stat_den"
[1] "Clinic_den"
[1] "high_rat"
[1] "mid_rat"
[1] "low_rat"
[1] "M_F_RAT"
[1] "P_DEN"
[1] "YOUN_DEP"
[1] "OLD_DEP"
[1] "AGING_IDX"
[1] "Income_mea"
[1] "Income_sta"


#### Correlation 

In [34]:
corr = cor(Variable_KS_df[c(col_y,col_X)])
# col_income = c( "Income_mea","Income_med" ,"Income_Q1", 
#                 "Income_Q3",  "Income_sta", "Income_CV")
# corr_income = sort(corr[col_income,'Nt'])
idx = abs(corr[,'Nt'])>.2
col_X_02 = names(corr[idx,'Nt']) %>% tail(-1)

In [35]:
(corr[idx,'Nt']) %>% tail(-1) %>% sort
print(col_X_02)

P_DEN   high_rat Income_mea Income_sta    OLD_DEP Clinic_den Temple_den 
-0.5689474 -0.3271878 -0.3221309 -0.3088155 -0.2962396 -0.2813614 -0.2479040 
   low_rat    M_F_RAT    Cluster 
 0.3520512  0.3697244  0.3828319

 [1] "Cluster"    "Temple_den" "Clinic_den" "high_rat"   "low_rat"   
 [6] "M_F_RAT"    "P_DEN"      "OLD_DEP"    "Income_mea" "Income_sta"


#### Global PCA

In [36]:
formula_Nt = Nt ~.

In [42]:
pca <- prcomp(formula_Nt,  #選擇變數 
              data = Variable_KS_df[c(col_X_02,col_y)],  # 資料
              scale = TRUE)                          # 正規化資料

ERROR: Error in prcomp.formula(formula_Nt, data = Variable_KS_df[c(col_X_02, : response not allowed in formula


In [39]:
class(Variable_KS_df[c(col_X_02,col_y)])

[1] "data.frame"

### GW PCA
-  [link01](https://gis.stackexchange.com/questions/35159/how-can-i-conduct-geographically-weighted-principal-component-analysis-using-arc)
- [運算方式](https://cihh.utp.ac.pa/sites/default/files/documentos/2021/pdf/geographically_weighted_principal_component.pdf)

In [ ]:
DM<-gw.dist(dp.locat=data.matrix(((Variable_KS_df[c('X', "Y")]))))


bw.gwpca.basic <- 
bw.gwpca(Variable_KS_shp, vars = col_X_02, k =5, robust = FALSE, adaptive = TRUE, dMat = DM)
# bw.gwpca.robust <- 
# bw.gwpca(Variable_KS_shp, vars = col_X_02, k = 5, robust = TRUE, adaptive = TRUE, dMat = DM )

In [ ]:
bw.gwpca.basic

In [ ]:
gwpca.basic <- gwpca(Variable_KS_shp,
 vars = col_X_02, bw = bw.gwpca.basic, k = 8, robust = FALSE, adaptive = TRUE)

### GLM_POISSON 

In [ ]:
Fit_Po <-glm(Nt~.,data=Variable_KS_df[,c(col_y,col_X_02)],family=poisson())
# Fit_Po <- glm(Nt~., data=Variable_KS_df[,c(col_y,col_X_02)], family = Gamma(link = "log"))
summary(Fit_Po) #查看回归模型参数


In [ ]:
# pdf_plot(Fit_Po$residuals)+xlab('residuals')
Variable_KS_centroid %>% 
  ggplot()+geom_sf(aes(color = residuals(Fit_Po), size = residuals(Fit_Po), alpha = .8))+
  scale_fill_gradient(low = "#56B1F7", high = "#132B43", na.value = NA)+
  theme_bw()

In [ ]:
nb <- poly2nb(Variable_KS_shp, queen=TRUE)
lw <- nb2listw(nb, style="W", zero.policy=TRUE)

In [ ]:
print(lw$weights[[1]])

### Spatial glmm
- [Package](https://kimura.univ-montp2.fr/~rousset/spaMM/spaMMintro.pdf)
- [Theory](https://bookdown.org/xiangyun/Thesis-Template-Bookdown/)

### GWR-Poisson

In [20]:
formula.GWPR = Nt ~.
DM<-gw.dist(dp.locat=data.matrix(((Variable_KS_df[c('X', "Y")]))))

In [22]:
bw <- bw.ggwr(formula.GWPR,  
                  data = Variable_KS_shp[c(col_X_02, col_y)],
                  family = "poisson",
                  approach = "AICc",
                  kernel = "gaussian", 
                  adaptive = TRUE,
                  dMat = DM )
bw

 Iteration    Log-Likelihood(With bandwidth:  43 )
       0       -336.3 
       1       -343.6 
       2       -268.2 
       3       -245.8 
       4       -242.3 
       5       -241.8 
       6       -241.7 
       7       -241.7 
Adaptive bandwidth (number of nearest neighbours): 43 AICc value: 523.6588 
 Iteration    Log-Likelihood(With bandwidth:  35 )
       0       -323.1 
       1       -327.3 
       2       -260.8 
       3       -235.6 
       4       -230.8 
       5       -229.4 
       6       -229.2 
       7       -229.2 
       8       -229.2 
Adaptive bandwidth (number of nearest neighbours): 35 AICc value: 507.702 
 Iteration    Log-Likelihood(With bandwidth:  28 )
       0       -313.8 
       1       -325.5 
       2       -258.2 
       3       -228.2 
       4       -222.1 
       5       -220.1 
       6       -219.7 
       7       -219.6 
       8       -219.6 
       9       -219.6 
Adaptive bandwidth (number of nearest neighbours): 28 AICc value: 497.6845 

[1] 22

In [ ]:
GWPR_model <- ggwr.basic(formula.GWPR, 
                       data = Variable_KS_shp[c(col_X_02, col_y)],
                       family = "poisson",
                       bw = bw, 
                       kernel = "gaussian", 
                       adaptive = TRUE,
                       dMat = DM)

In [ ]:
GWPR_model$glms$residuals %>% pdf_plot() +
xlab("residual of GWPR")


In [ ]:
ggplot()+geom_point(aes(x = Variable_KS_df$X, y = Variable_KS_df$Y, color = GWPR_model$glms$residual, size = 2))